In [1]:
import pandas as pd
import numpy as np

In [10]:
attributes_data=pd.read_csv(r'C:/Users/Jaketosis/OneDrive/TAMU/MS Stats/STAT626/Project/Data/Attributes_DataFrame.csv')
daily_data=pd.read_csv(r'C:/Users/Jaketosis/OneDrive/TAMU/MS Stats/STAT626/Project/Data/Daily_DataFrame.csv')

In [11]:
daily_data

,Movie_Title,Date,Daily,Theaters,Rank
0,Jurassic World (2015),2015-06-12,81953950,4274.0,1
1,Jurassic World (2015),2015-06-13,69644830,4274.0,1
2,Jurassic World (2015),2015-06-14,57207490,4274.0,1
3,Jurassic World (2015),2015-06-15,25344820,4274.0,1
4,Jurassic World (2015),2015-06-16,24342515,4274.0,1
...,...,...,...,...,...
254057,A Call to Spy (2020),2020-11-17,623,8.0,17
254058,A Call to Spy (2020),2020-11-18,568,8.0,17
254059,A Call to Spy (2020),2020-11-19,597,8.0,17
254060,Love Story (2020),2020-02-09,87198,650.0,20


In [12]:
attributes_data

,Title,Domestic,International,Budget,Distributor,MPAA-Rating,Runtime,Genres
0,Jurassic World (2015),652270625,1018130012,150000000.0,Universal Pictures,PG-13,124,Action;Adventure;Sci-Fi
1,Star Wars: Episode VII - The Force Awakens (2015),936662225,1131561399,245000000.0,Walt Disney Studios Motion Pictures,PG-13,138,Action;Adventure;Sci-Fi
2,Avengers: Age of Ultron (2015),459005868,943800000,250000000.0,Walt Disney Studios Motion Pictures,PG-13,141,Action;Adventure;Sci-Fi
3,Inside Out (2015),356461711,501149463,175000000.0,Walt Disney Studios Motion Pictures,PG,95,Adventure;Animation;Comedy;Drama;Family;Fantasy
4,Furious 7 (2015),353007020,1162040651,190000000.0,Universal Pictures,PG-13,137,Action;Adventure;Thriller
...,...,...,...,...,...,...,...,...
3395,The Wolf of Snow Hollow (2020),185026,81937,NaN,United Artists Releasing,R,83,Comedy;Horror;Thriller
3396,Come Away (2020),184477,1480982,NaN,Relativity Media,PG,94,Adventure;Drama;Fantasy
3397,No Escape (2020),167962,4135870,NaN,Vertical Entertainment,R,91,Adventure;Horror;Mystery;Thriller
3398,A Call to Spy (2020),159014,681026,NaN,IFC Films,PG-13,123,Biography;Crime;Drama;Thriller;War


In [ ]:
attraibutes